In [64]:
import numpy as np
import pandas as pd


In [65]:
import requests
import zipfile
import io

# URL of the API endpoint that provides the zip file
api_url = "https://api.data.gov.my/gtfs-static/ktmb"

try:
    # Make a GET request to the API
    response = requests.get(api_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the zip file from the response content
        with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
            # Extract all contents to a directory
            zip_ref.extractall("ktmb-data")
        print("Data extracted successfully.")
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
except Exception as e:
    print("An error occurred:", e)


Data extracted successfully.


In [66]:
# pip install gtfs-realtime-bindings pandas requests
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import pandas as pd
from requests import get
 
# Sample GTFS-R URL from Malaysia's Open API
URL = 'https://api.data.gov.my/gtfs-realtime/vehicle-position/ktmb'
 
# Parse the GTFS Realtime feed
feed = gtfs_realtime_pb2.FeedMessage()
response = get(URL)
feed.ParseFromString(response.content)
 
# Extract and print vehicle position information
vehicle_positions = [MessageToDict(entity.vehicle) for entity in feed.entity]
print(f'Total vehicles: {len(vehicle_positions)}')
df = pd.json_normalize(vehicle_positions)
print(df.head())


Total vehicles: 21
    timestamp   trip.tripId  position.latitude  position.longitude  \
0  1708329987          9420           6.225504           100.36257   
1  1708330006          2943           5.625558           100.49347   
2  1708330008          9174           5.390114           100.37060   
3  1708330007  weekday_2013           3.062471           101.70768   
4  1708330007  weekday_2011           3.190069           101.68800   

   position.bearing  position.speed    vehicle.id vehicle.label  
0             337.5       139.17668          9420     ETS 93202  
1             337.5       129.17300          2943      EMU 8324  
2             135.0        20.00000          9174     ETS 93213  
3             180.0        92.34122  weekday_2013      SCS 9211  
4               0.0         1.00000  weekday_2011      EMU 8105  


In [67]:
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')

C:\Users\nasuha.imanina\AppData\Local\Temp\ipykernel_15520\20987537.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')


In [68]:
df

,timestamp,trip.tripId,position.latitude,position.longitude,position.bearing,position.speed,vehicle.id,vehicle.label
0,2024-02-19 08:06:24,9420,6.225504,100.362570,337.5,139.176680,9420,ETS 93202
1,2024-02-19 08:06:24,2943,5.625558,100.493470,337.5,129.173000,2943,EMU 8324
2,2024-02-19 08:06:24,9174,5.390114,100.370600,135.0,20.000000,9174,ETS 93213
3,2024-02-19 08:06:24,weekday_2013,3.062471,101.707680,180.0,92.341220,weekday_2013,SCS 9211
4,2024-02-19 08:06:24,weekday_2011,3.190069,101.688000,0.0,1.000000,weekday_2011,EMU 8105
5,2024-02-19 08:06:24,weekday_2139,3.084648,101.622510,0.0,1.000000,weekday_2139,SCS 9203
6,2024-02-19 08:06:24,2906,5.042821,100.622795,270.0,129.173000,2906,EMU 8340
7,2024-02-19 08:06:24,weekday_2014,3.185788,101.689285,0.0,1.000000,weekday_2014,EMU 8325
8,2024-02-19 08:06:24,weekday_2115,3.084540,101.631600,270.0,13.000000,weekday_2115,SCS 9238
9,2024-02-19 08:06:24,weekday_2029,2.753963,101.826775,292.5,40.649370,weekday_2029,SCS 9237


In [69]:
stoptime = pd.read_csv('ktmb-data/stop_times.txt')
trip = pd.read_csv('ktmb-data/trips.txt')
route = pd.read_csv('ktmb-data/routes.txt')
stop = pd.read_csv('ktmb-data/stops.txt')

In [70]:
newdata = pd.concat([df,trip],join='inner',axis=1)
newdata

,timestamp,trip.tripId,position.latitude,position.longitude,position.bearing,position.speed,vehicle.id,vehicle.label,route_id,service_id,trip_id,direction_id
0,2024-02-19 08:06:24,9420,6.225504,100.362570,337.5,139.176680,9420,ETS 93202,KC05_KB18,komuter_weekday,weekday_2003,0
1,2024-02-19 08:06:24,2943,5.625558,100.493470,337.5,129.173000,2943,EMU 8324,KC05_KB18,komuter_weekday,weekday_2007,0
2,2024-02-19 08:06:24,9174,5.390114,100.370600,135.0,20.000000,9174,ETS 93213,KC05_KB18,komuter_weekday,weekday_2011,0
3,2024-02-19 08:06:24,weekday_2013,3.062471,101.707680,180.0,92.341220,weekday_2013,SCS 9211,KC05_KB18,komuter_weekday,weekday_2013,0
4,2024-02-19 08:06:24,weekday_2011,3.190069,101.688000,0.0,1.000000,weekday_2011,EMU 8105,KC05_KB18,komuter_weekday,weekday_2015,0
5,2024-02-19 08:06:24,weekday_2139,3.084648,101.622510,0.0,1.000000,weekday_2139,SCS 9203,KC05_KB18,komuter_weekday,weekday_2017,0
6,2024-02-19 08:06:24,2906,5.042821,100.622795,270.0,129.173000,2906,EMU 8340,KC05_KB18,komuter_weekday,weekday_2019,0
7,2024-02-19 08:06:24,weekday_2014,3.185788,101.689285,0.0,1.000000,weekday_2014,EMU 8325,KC05_KB18,komuter_weekday,weekday_2021,0
8,2024-02-19 08:06:24,weekday_2115,3.084540,101.631600,270.0,13.000000,weekday_2115,SCS 9238,KC05_KB18,komuter_weekday,weekday_2023,0
9,2024-02-19 08:06:24,weekday_2029,2.753963,101.826775,292.5,40.649370,weekday_2029,SCS 9237,KC05_KB18,komuter_weekday,weekday_2025,0


In [71]:
newdata = pd.concat([newdata,route],axis=1,join='inner')
newdata

,timestamp,trip.tripId,position.latitude,position.longitude,position.bearing,position.speed,vehicle.id,vehicle.label,route_id,service_id,...,direction_id,agency_id,route_id,route_type,route_short_name,route_long_name,route_desc,route_url,route_color,route_text_color
0,2024-02-19 08:06:24,9420,6.225504,100.362570,337.5,139.17668,9420,ETS 93202,KC05_KB18,komuter_weekday,...,0,ktmb,KC05_KB18,0,Seremban Line,KTM Batu Caves - Pulau Sebang/Tampin,NaN,https://www.ktmb.com.my/komuter-routes-map.html,3C5A9F,FFFFFF
1,2024-02-19 08:06:24,2943,5.625558,100.493470,337.5,129.17300,2943,EMU 8324,KC05_KB18,komuter_weekday,...,0,ktmb,KA15_KD19,0,Port Klang Line,KTM Tanjung Malim - Pelabuhan Klang,NaN,https://www.ktmb.com.my/komuter-routes-map.html,DC2420,FFFFFF
2,2024-02-19 08:06:24,9174,5.390114,100.370600,135.0,20.00000,9174,ETS 93213,KC05_KB18,komuter_weekday,...,0,ktmb,100_47300,0,Padang Besar Line,KTM Butterworth - Padang Besar,NaN,https://www.ktmb.com.my/komuter-routes-map.html,018000,FFFFFF
3,2024-02-19 08:06:24,weekday_2013,3.062471,101.707680,180.0,92.34122,weekday_2013,SCS 9211,KC05_KB18,komuter_weekday,...,0,ktmb,100_9000,0,Ipoh Line,KTM Butterworth - Ipoh,NaN,https://www.ktmb.com.my/komuter-routes-map.html,1964B7,FFFFFF
4,2024-02-19 08:06:24,weekday_2011,3.190069,101.688000,0.0,1.00000,weekday_2011,EMU 8105,KC05_KB18,komuter_weekday,...,0,ktmb,SH,2,SH,Intercity Shuttle Tumpat - Gemas,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN
5,2024-02-19 08:06:24,weekday_2139,3.084648,101.622510,0.0,1.00000,weekday_2139,SCS 9203,KC05_KB18,komuter_weekday,...,0,ktmb,ERT,2,ERT,Intercity Ekspres Rakyat Timuran Tumpat - JB S...,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN
6,2024-02-19 08:06:24,2906,5.042821,100.622795,270.0,129.17300,2906,EMU 8340,KC05_KB18,komuter_weekday,...,0,ktmb,ES,2,ES,Intercity Ekspres Selatan Gemas - JB Sentral,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN
7,2024-02-19 08:06:24,weekday_2014,3.185788,101.689285,0.0,1.00000,weekday_2014,EMU 8325,KC05_KB18,komuter_weekday,...,0,ktmb,ST,2,ST,Intercity Shuttle Tebrau JB Sentral - Woodlands,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN
8,2024-02-19 08:06:24,weekday_2115,3.084540,101.631600,270.0,13.00000,weekday_2115,SCS 9238,KC05_KB18,komuter_weekday,...,0,ktmb,ETS,2,ETS,Electric Train Service Padang Besar - Gemas,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN


In [72]:
newdata = pd.concat([newdata,stop],axis=1, join='inner')
newdata

,timestamp,trip.tripId,position.latitude,position.longitude,position.bearing,position.speed,vehicle.id,vehicle.label,route_id,service_id,...,route_short_name,route_long_name,route_desc,route_url,route_color,route_text_color,stop_id,stop_name,stop_lat,stop_lon
0,2024-02-19 08:06:24,9420,6.225504,100.362570,337.5,139.17668,9420,ETS 93202,KC05_KB18,komuter_weekday,...,Seremban Line,KTM Batu Caves - Pulau Sebang/Tampin,NaN,https://www.ktmb.com.my/komuter-routes-map.html,3C5A9F,FFFFFF,50500,Taman Wahyu,3.214510,101.672178
1,2024-02-19 08:06:24,2943,5.625558,100.493470,337.5,129.17300,2943,EMU 8324,KC05_KB18,komuter_weekday,...,Port Klang Line,KTM Tanjung Malim - Pelabuhan Klang,NaN,https://www.ktmb.com.my/komuter-routes-map.html,DC2420,FFFFFF,50600,BATU CAVES,3.237796,101.681215
2,2024-02-19 08:06:24,9174,5.390114,100.370600,135.0,20.00000,9174,ETS 93213,KC05_KB18,komuter_weekday,...,Padang Besar Line,KTM Butterworth - Padang Besar,NaN,https://www.ktmb.com.my/komuter-routes-map.html,018000,FFFFFF,52800,Angkasapuri,3.113212,101.673367
3,2024-02-19 08:06:24,weekday_2013,3.062471,101.707680,180.0,92.34122,weekday_2013,SCS 9211,KC05_KB18,komuter_weekday,...,Ipoh Line,KTM Butterworth - Ipoh,NaN,https://www.ktmb.com.my/komuter-routes-map.html,1964B7,FFFFFF,52900,Pantai Dalam,3.095607,101.669927
4,2024-02-19 08:06:24,weekday_2011,3.190069,101.688000,0.0,1.00000,weekday_2011,EMU 8105,KC05_KB18,komuter_weekday,...,SH,Intercity Shuttle Tumpat - Gemas,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN,53000,PETALING,3.086485,101.664338
5,2024-02-19 08:06:24,weekday_2139,3.084648,101.622510,0.0,1.00000,weekday_2139,SCS 9203,KC05_KB18,komuter_weekday,...,ERT,Intercity Ekspres Rakyat Timuran Tumpat - JB S...,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN,47300,PADANG BESAR,6.662205,100.319514
6,2024-02-19 08:06:24,2906,5.042821,100.622795,270.0,129.17300,2906,EMU 8340,KC05_KB18,komuter_weekday,...,ES,Intercity Ekspres Selatan Gemas - JB Sentral,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN,54500,Bukit Badak,3.036147,101.470176
7,2024-02-19 08:06:24,weekday_2014,3.185788,101.689285,0.0,1.00000,weekday_2014,EMU 8325,KC05_KB18,komuter_weekday,...,ST,Intercity Shuttle Tebrau JB Sentral - Woodlands,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN,54700,KLANG,3.043078,101.449543
8,2024-02-19 08:06:24,weekday_2115,3.084540,101.631600,270.0,13.00000,weekday_2115,SCS 9238,KC05_KB18,komuter_weekday,...,ETS,Electric Train Service Padang Besar - Gemas,NaN,https://www.ktmb.com.my/intercity-routes-map.html,NaN,NaN,50400,Kampung Batu,3.204892,101.675584


In [73]:
newdata = pd.concat([newdata, stoptime], join='inner',axis=1)
newdata

,timestamp,trip.tripId,position.latitude,position.longitude,position.bearing,position.speed,vehicle.id,vehicle.label,route_id,service_id,...,stop_id,stop_name,stop_lat,stop_lon,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled
0,2024-02-19 08:06:24,9420,6.225504,100.362570,337.5,139.17668,9420,ETS 93202,KC05_KB18,komuter_weekday,...,50500,Taman Wahyu,3.214510,101.672178,26,20:25:00,20:25:00,37500,100,0.00
1,2024-02-19 08:06:24,2943,5.625558,100.493470,337.5,129.17300,2943,EMU 8324,KC05_KB18,komuter_weekday,...,50600,BATU CAVES,3.237796,101.681215,26,20:29:00,20:29:00,37405,200,2.70
2,2024-02-19 08:06:24,9174,5.390114,100.370600,135.0,20.00000,9174,ETS 93213,KC05_KB18,komuter_weekday,...,52800,Angkasapuri,3.113212,101.673367,26,20:31:00,20:31:00,37400,300,4.50
3,2024-02-19 08:06:24,weekday_2013,3.062471,101.707680,180.0,92.34122,weekday_2013,SCS 9211,KC05_KB18,komuter_weekday,...,52900,Pantai Dalam,3.095607,101.669927,26,20:36:00,20:36:00,37200,400,7.50
4,2024-02-19 08:06:24,weekday_2011,3.190069,101.688000,0.0,1.00000,weekday_2011,EMU 8105,KC05_KB18,komuter_weekday,...,53000,PETALING,3.086485,101.664338,26,20:42:00,20:44:00,36900,500,11.30
5,2024-02-19 08:06:24,weekday_2139,3.084648,101.622510,0.0,1.00000,weekday_2139,SCS 9203,KC05_KB18,komuter_weekday,...,47300,PADANG BESAR,6.662205,100.319514,26,21:01:00,21:01:00,36600,700,23.70
6,2024-02-19 08:06:24,2906,5.042821,100.622795,270.0,129.17300,2906,EMU 8340,KC05_KB18,komuter_weekday,...,54500,Bukit Badak,3.036147,101.470176,26,21:14:00,21:16:00,36000,900,32.20
7,2024-02-19 08:06:24,weekday_2014,3.185788,101.689285,0.0,1.00000,weekday_2014,EMU 8325,KC05_KB18,komuter_weekday,...,54700,KLANG,3.043078,101.449543,26,21:28:00,21:28:00,35500,1000,42.10
8,2024-02-19 08:06:24,weekday_2115,3.084540,101.631600,270.0,13.00000,weekday_2115,SCS 9238,KC05_KB18,komuter_weekday,...,50400,Kampung Batu,3.204892,101.675584,26,21:46:00,21:46:00,34800,1100,55.95


In [74]:
newdata.drop(['stop_id','trip_id'],axis=1,inplace=True)


In [75]:
newdata.columns

Index(['timestamp', 'trip.tripId', 'position.latitude', 'position.longitude',
       'position.bearing', 'position.speed', 'vehicle.id', 'vehicle.label',
       'route_id', 'service_id', 'direction_id', 'agency_id', 'route_id',
       'route_type', 'route_short_name', 'route_long_name', 'route_desc',
       'route_url', 'route_color', 'route_text_color', 'stop_name', 'stop_lat',
       'stop_lon', 'arrival_time', 'departure_time', 'stop_sequence',
       'shape_dist_traveled'],
      dtype='object')

In [76]:
newdata['timestamp'] = newdata['timestamp'].astype(str)

In [78]:
from keplergl import KeplerGl
from datetime import datetime
import json

# Assuming the 'arrival_time' column represents the time dimension
# You can also use 'departure_time' or any other time-related column from your GTFS data

# Create a Kepler.gl map
map_1 = KeplerGl()

# Add data to the map
# map_1.add_data(data=station[station['coordinates']], name='coordinate')
# Convert Timestamps to strings
# newdata['timestamp'] = newdata['timestamp'].astype(str)

# # Serialize DataFrame to JSON
# json_data = newdata.to_json(orient='records')
# print(json_data)

map_1.add_data(data=newdata, name='ktmb')


# Configure the Kepler.gl map to animate based on the timestamp column
map_1.config = {
    'version': 'v1',
    'config': {
        "mapState":{
            "latitude": 3.140853, 
            "longitude": 101.693207, 
            "zoom": 5
            }
    }
}


# Visualize the animated data within Kepler.gl

map_1
# You can further configure the map interactively in the Jupyter Notebook environment

# If you want to save the map configuration for later use or share it with others
# you can save it as a JSON file
# config = map_1.config

# # config["config"]["animationConfig"]["currentTime"] = str(config["config"]["animationConfig"]["currentTime"])

# configsave = json.dumps(config, default=str)
# print(configsave)
# with open('keplergl_config.json', 'w') as f:
#     f.write(configsave)

# # You can also export the map as an HTML file
# map_1.save_to_html(file_name='ktmb-data.html')
# map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\nasuha.imanina\Documents\railway-test\venv\lib\site-packages\keplergl\keplergl.py:22: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df.to_dict('split')
c:\Users\nasuha.imanina\Documents\railway-test\venv\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 3.140853, 'longitude': 101.693207, 'zoom…

In [79]:
config = map_1.config


In [80]:
configsavektmb = json.dumps(config, default=str)
print(configsavektmb)
with open('keplergl_config-ktmb.json', 'w') as f:
    f.write(configsavektmb)

# You can also export the map as an HTML file
map_1.save_to_html(file_name='ktmb-data.html')

{"version": "v1", "config": {"visState": {"filters": [], "layers": [{"id": "qetfe7", "type": "point", "config": {"dataId": "ktmb", "label": "position", "color": [255, 153, 31], "highlightColor": [252, 242, 26, 255], "columns": {"lat": "position.latitude", "lng": "position.longitude", "altitude": null}, "isVisible": true, "visConfig": {"radius": 10, "fixedRadius": false, "opacity": 0.8, "outline": false, "thickness": 2, "strokeColor": null, "colorRange": {"name": "Global Warming", "type": "sequential", "category": "Uber", "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]}, "strokeColorRange": {"name": "Global Warming", "type": "sequential", "category": "Uber", "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]}, "radiusRange": [0, 50], "filled": true}, "hidden": false, "textLabel": [{"field": null, "color": [255, 255, 255], "size": 18, "offset": [0, 0], "anchor": "start", "alignment": "center"}]}, "visualChannels": {"colorField": {"na